In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from pandas import Timestamp
import csv
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [3]:
custom_date_parser=lambda dates: [ datetime.strptime(str(d), "%Y-%m-%d") for d in dates]  
# for datetime "%Y-%m-%d %H:%M:%S"
dfraw = pd.read_csv('data.csv', quoting=csv.QUOTE_NONNUMERIC,quotechar='"',sep = ',', header = 0, index_col = None)
dfraw["date_trans"]=pd.to_datetime(dfraw["date_trans"],format="%Y-%m-%d")
dfraw

,id,ref,date_trans,status,amount
0,1.0,INV-220001,2022-01-01,OPEN,35.94
1,2.0,INV-220002,2022-01-02,OPEN,99.99
2,3.0,INV-220003,2022-01-03,CANCELLED,13.00
3,4.0,INV-220003,2022-01-04,OPEN,NaN
4,5.0,INV-220005,2022-01-04,OPEN,204.20
5,6.0,INV-220006,2022-01-15,OPEN,-4.20
6,7.0,NaN,2022-01-06,NaN,0.00
7,8.0,INV-220007,2022-01-15,PENDING,50.40
8,9.0,NaN,NaT,ERROR,0.00
9,10.0,INV-220007,2022-01-15,PENDING,50.40


In [4]:
def dataframe_to_code(df):
    data = np.array2string(df.to_numpy(), separator=', ')
    data = data.replace(" nan", " float('nan')")
    data = data.replace(" NaT", " pd.NaT")

    cols = df.columns.tolist()
    return f"""df = pd.DataFrame({data}, columns={cols})"""
#print(dataframe_to_code(dfraw))
#use this to create the code for dataframes

In [5]:
df = pd.DataFrame([
    [1, 'INV-220001', Timestamp('2022-01-01 00:00:00'), 'OPEN', 35.94,''],
    [2, 'INV-220002', Timestamp('2022-01-02 00:00:00'), 'OPEN', 99.99,'-5'],
    [3, 'INV-220003', Timestamp('2022-01-03 00:00:00'), 'CANCELLED', 13.0,'reinbursed 10.5'],
    [4, 'INV-220003', Timestamp('2022-01-04 00:00:00'), 'OPEN', float('nan'),''],
    [5, 'INV-220005', Timestamp('2022-01-04 00:00:00'), 'OPEN', 204.2,''],
    [6, 'INV-220006', Timestamp('2022-01-15 00:00:00'), 'OPEN', -4.2,'discount'],
    [7, float('nan'), Timestamp('2022-01-06 00:00:00'), float('nan'), 0.0,''],
    [8, 'INV-220007', Timestamp('2022-01-15 00:00:00'), 'PENDING', 50.4,''],
    [9, '', pd.NaT, 'ERROR', 0.0,''],
    [10, 'INV-220007', Timestamp('2022-01-15 00:00:00'), 'PENDING', 50.4,'']], 
    columns=['id', 'ref', 'date_trans', 'status', 'amount','notes'])
df

,id,ref,date_trans,status,amount,notes
0,1,INV-220001,2022-01-01,OPEN,35.94,
1,2,INV-220002,2022-01-02,OPEN,99.99,-5
2,3,INV-220003,2022-01-03,CANCELLED,13.00,reinbursed 10.5
3,4,INV-220003,2022-01-04,OPEN,NaN,
4,5,INV-220005,2022-01-04,OPEN,204.20,
5,6,INV-220006,2022-01-15,OPEN,-4.20,discount
6,7,NaN,2022-01-06,NaN,0.00,
7,8,INV-220007,2022-01-15,PENDING,50.40,
8,9,,NaT,ERROR,0.00,
9,10,INV-220007,2022-01-15,PENDING,50.40,


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          10 non-null     int64         
 1   ref         9 non-null      object        
 2   date_trans  9 non-null      datetime64[ns]
 3   status      9 non-null      object        
 4   amount      9 non-null      float64       
 5   notes       10 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 608.0+ bytes


In [7]:
def check(in_df):
    df=in_df.copy()
    dtypes = df.dtypes.to_dict()
    col_metrics=[]
    for col, typ in dtypes.items():
        print (col,typ)
        metrics={}
        metrics["column"]=col
        metrics["dtype"]=typ
        metrics["records"]=len(df[col])
        metrics["count_unique"]=len(set(df[pd.notna(df[col])][col]))
        metrics["nans"]=len(df[pd.isnull(df[col])])
        if metrics["count_unique"]<5:
            value_counts_series=df[col].value_counts(dropna=False)
            metrics["value_counts"]=value_counts_series.to_dict()
        else:
            metrics["value_counts"]=[]
        if "float" in str(typ):
            print("float")
            metrics["max"]=max(df[col])
            metrics["min"]=min(df[col])    
            metrics["sum"]=sum(df[col])    
            metrics["sum_abs"]=sum(abs(df[col]))
            metrics['std']=df[col].std()
            metrics["mode"]=df[col].mode() 
        elif "int" in str(typ):
            metrics["max"]=max(df[col])
            metrics["min"]=min(df[col])    
            metrics["sum"]=sum(df[col])    
            metrics["sum_abs"]=sum(abs(df[col]))
            metrics['std']=df[col].std()
            metrics["mode"]=list(df[col].mode())
        elif is_datetime(df[col]):
            metrics["max"]=max(df[col])
            metrics["min"]=min(df[col])    
            
            
        elif typ=="object":
            values=df[pd.notna(df[col])][col]
            numeric_chars=values.str.replace(r'[^0-9^-^.]+', '',regex=True)
            numeric_chars_no_blank= numeric_chars[numeric_chars.str.len()>0]
            numeric=pd.to_numeric(numeric_chars_no_blank, errors='coerce')
            if len(numeric)>0:
                metrics["max"]=max(numeric)
                metrics["min"]=min(numeric)
            metrics["empty_strings"]=len(df[df[col].eq('')])
            
        #print(metrics)
        col_metrics.append(metrics)
    return pd.DataFrame(col_metrics)     
        
check(df)

id int64
ref object
date_trans datetime64[ns]
status object
amount float64
float
notes object


,column,dtype,records,count_unique,nans,value_counts,max,min,sum,sum_abs,std,mode,empty_strings
0,id,int64,10,10,0,[],10,1,55.0,55.0,3.027650,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",NaN
1,ref,object,10,7,1,[],220007,220001,NaN,NaN,NaN,NaN,1.0
2,date_trans,datetime64[ns],10,6,1,[],2022-01-15 00:00:00,2022-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
3,status,object,10,4,1,"{'OPEN': 5, 'PENDING': 2, 'CANCELLED': 1, 'ERR...",NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,amount,float64,10,7,1,[],204.2,-4.2,NaN,NaN,66.877944,0 0.0 1 50.4 dtype: float64,NaN
5,notes,object,10,4,0,"{'': 7, 'discount': 1, 'reinbursed 10.5': 1, '...",10.5,5,NaN,NaN,NaN,NaN,7.0


In [8]:
def clean_cols(in_df,date_fillna="latest"):
    df=in_df.copy()
    dtypes = df.dtypes.to_dict()
    for col, typ in dtypes.items():
        if typ=="float" or typ=="int64":
            df[col].fillna(0,inplace=True)        
        elif is_datetime(df[col]):
            if date_fillna=="latest":
                val=max(df[col])
            elif date_fillna=="first":
                val=min(df[col])
            elif  isinstance(date_fillna, datetime):
                val=date_fillna
            df[col].fillna(val,inplace=True)
        elif typ=="object":
            df[col].fillna('',inplace=True)
    return df    
n=datetime.now()    
clean_cols(df,date_fillna=n)

,id,ref,date_trans,status,amount,notes
0,1,INV-220001,2022-01-01 00:00:00.000000,OPEN,35.94,
1,2,INV-220002,2022-01-02 00:00:00.000000,OPEN,99.99,-5
2,3,INV-220003,2022-01-03 00:00:00.000000,CANCELLED,13.00,reinbursed 10.5
3,4,INV-220003,2022-01-04 00:00:00.000000,OPEN,0.00,
4,5,INV-220005,2022-01-04 00:00:00.000000,OPEN,204.20,
5,6,INV-220006,2022-01-15 00:00:00.000000,OPEN,-4.20,discount
6,7,,2022-01-06 00:00:00.000000,,0.00,
7,8,INV-220007,2022-01-15 00:00:00.000000,PENDING,50.40,
8,9,,2022-01-26 13:59:23.176813,ERROR,0.00,
9,10,INV-220007,2022-01-15 00:00:00.000000,PENDING,50.40,


In [8]:
def check_sequence(in_df):
    df=in_df.copy()
    dtypes = df.dtypes.to_dict()
    for col, typ in dtypes.items():
        print(col,typ)
        if "int" in str(typ):
            unique=  set([i for i in set(df[pd.notna(df[col])][col])])
            fullrng= range(min(unique),max(unique)+1)
            if unique.issubset(fullrng):
                print("Full sequence")
            else:
                diff=unique.difference(fullrng)
                print("Missing in sequence: ",list(diff)[0:10])
            
                
        elif is_datetime(df[col]):
            unique =set([i.date() for i in set(df[pd.notna(df[col])][col])])
            fullrng=set(pd.date_range(min(unique),max(unique)-timedelta(days=1),freq='d').to_list())
            if unique.issubset(fullrng):
                print("Full sequence")
            else:
                diff=unique.difference(fullrng)
                print(len(diff)," missing in sequence, first ",min(len(diff),10),":",list(diff)[0:10])
        elif typ=="object":
            values=df[pd.notna(df[col])][col]
            numeric_chars=values.str.replace(r'[^0-9^-^.]+', '',regex=True)
            numeric_chars_no_blank= numeric_chars[numeric_chars.str.len()>0]
            numeric=pd.to_numeric(numeric_chars_no_blank, errors='coerce',downcast='integer')
            if pd.api.types.is_float_dtype(numeric):
                #we have floats so it wont likely be a sequence
                print("Contains floats, no sequence to check")
            else:
                unique=set(numeric)
                print(list(unique))
                if unique:
                    
                    fullrng=set(range(min(unique),max(unique)))
                    if unique.issubset(fullrng):
                        print("Full sequence")
                    else:
                        diff=unique.difference(fullrng)
                        print(len(diff)," missing in sequence, fist 10:",list(diff)[0:10])           
                else:
                    print("No sequence to check")
    return

check_sequence(df)

id int64
Full sequence
ref object
[220001, 220002, 220003, 220005, 220006, 220007]
1  missing in sequence, fist 10: [220007]
date_trans datetime64[ns]
6  missing in sequence, first  6 : [datetime.date(2022, 1, 2), datetime.date(2022, 1, 3), datetime.date(2022, 1, 1), datetime.date(2022, 1, 6), datetime.date(2022, 1, 4), datetime.date(2022, 1, 15)]
status object
[]
No sequence to check
amount float64
notes object
Contains floats, no sequence to check


In [9]:
def check_duplicates(df, columns=[],keep='first', ascending=None):
    cols=columns
    if not isinstance(df,pd.DataFrame):
        return
    if isinstance(cols,list) and len(columns)==0:
        cols=list(df.columns)
    if isinstance(columns,str):
        cols=[columns]
    if isinstance(cols,int):
        cols=[df.columns[columns]]
    df_duplicates=df[df.duplicated(subset=cols,keep=keep)]
    if not columns:
        fields="entire record"
    else:
        fields=",".join(cols)
    print("Duplicates in",fields,"(keep=",keep,"):",len(df_duplicates)," of population: ", len(df))
    if len(df_duplicates)==0:
        return None
    else:
        if ascending is True:
            print("Ascending")
            return df_duplicates.sort_values(cols, ascending=True)
        elif ascending is False:
            print("Descending")
            return df_duplicates.sort_values(cols, ascending=False)
        else:
            print("No sort")
            return df_duplicates
    return 
check_duplicates(df,"ref", ascending=True)

Duplicates in ref (keep= first ): 2  of population:  10
Ascending


,id,ref,date_trans,status,amount,notes
3,4,INV-220003,2022-01-04,OPEN,NaN,
9,10,INV-220007,2022-01-15,PENDING,50.4,


In [10]:
clean_cols(df)

,id,ref,date_trans,status,amount,notes
0,1,INV-220001,2022-01-01,OPEN,35.94,
1,2,INV-220002,2022-01-02,OPEN,99.99,-5
2,3,INV-220003,2022-01-03,CANCELLED,13.00,reinbursed 10.5
3,4,INV-220003,2022-01-04,OPEN,0.00,
4,5,INV-220005,2022-01-04,OPEN,204.20,
5,6,INV-220006,2022-01-15,OPEN,-4.20,discount
6,7,,2022-01-06,,0.00,
7,8,INV-220007,2022-01-15,PENDING,50.40,
8,9,,2022-01-15,ERROR,0.00,
9,10,INV-220007,2022-01-15,PENDING,50.40,


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,10.0,5.50,3.027650,1.0,3.25,5.50,7.75,10.0
amount,9.0,49.97,66.877944,-4.2,0.00,35.94,50.40,204.2
